In [1]:
import cryptocompare
import pandas as pd
import datetime
import boto3
import datetime
import requests
import json
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
import re
%matplotlib inline
from datetime import datetime, timedelta
import awswrangler as wr
pd.set_option('display.max_columns', None)

import pytz

utc=pytz.UTC


CRYPTO_COMPARE_KEY = "6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4"
cryptocompare.cryptocompare._set_api_key_parameter(CRYPTO_COMPARE_KEY)

'&api_key=6188db038cad874de5eb7da3821490d45967ae07911bb365904d2b1759400ea4'

In [2]:
session = boto3.Session(profile_name='master')
s3_client = session.client('s3')
bucket = 'kosmobiker-masterproject'

In [3]:
def get_list_of_objects_s3(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')


In [4]:
def get_json_s3(bucket, key, s3=None):
    """
    Function is used to get JSON file
    from S3 bucket
    """
    if not s3:
        s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [5]:
params = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_crypto_data/"
        }

In [7]:
data = []
for key in get_list_of_objects_s3(params, s3_client):
    try:
        if key.endswith('.json'):
            coin_cur = re.findall("(?:[0-9]{4}_[0-9]{2}_[0-9]{2})_(.+)(?:.json)", key)[0]
            tmp = get_json_s3(bucket, key, s3_client)
            res = [dict(item, **{'coin_currency':f'{coin_cur}'}) for item in tmp]
            data.extend(res)
    except Exception as err:
        print(err)
#write to glue table

In [8]:
df_test = pd.DataFrame(data)

In [9]:
df_test['partition_column'] = pd.to_datetime(df_test['time'], unit='s').dt.strftime('%Y-%m-%d')
df_test

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,coin_currency,partition_column
0,1659054600,0.01162,0.01162,0.01162,60.63,0.7048,0.01162,direct,,BNB_BTC,2022-07-29
1,1659054660,0.01163,0.01162,0.01162,56.18,0.6535,0.01163,direct,,BNB_BTC,2022-07-29
2,1659054720,0.01164,0.01163,0.01163,42.43,0.4938,0.01164,direct,,BNB_BTC,2022-07-29
3,1659054780,0.01165,0.01164,0.01164,51.33,0.5979,0.01165,direct,,BNB_BTC,2022-07-29
4,1659054840,0.01166,0.01165,0.01165,98.17,1.1450,0.01166,direct,,BNB_BTC,2022-07-29
...,...,...,...,...,...,...,...,...,...,...,...
224791,1659399960,0.38110,0.38060,0.38110,78119.74,29740.5700,0.38060,direct,,XRP_USD,2022-08-02
224792,1659400020,0.38090,0.38060,0.38060,56706.23,21597.1300,0.38090,direct,,XRP_USD,2022-08-02
224793,1659400080,0.38100,0.38090,0.38090,37514.45,14298.2800,0.38100,direct,,XRP_USD,2022-08-02
224794,1659400140,0.38100,0.38100,0.38100,58546.00,22316.5300,0.38100,direct,,XRP_USD,2022-08-02


In [17]:
df_test.dtypes

time                  int64
high                float64
low                 float64
open                float64
volumefrom          float64
volumeto            float64
close               float64
conversiontype       object
conversionsymbol     object
coin_currency        object
partition_column     object
dtype: object

In [26]:
def write_to_glue(pandas_df: pd.DataFrame, path_table, database_name, table_name, col_partition):
    try:
        wr.s3.to_parquet(
            df=pandas_df,
            index=False,
            path=path_table,
            dataset=True,
            database=database_name,
            compression='snappy',
            table=table_name,
            mode="overwrite_partitions",
            partition_cols=col_partition,
            use_threads=True,
            concurrent_partitioning=True,
            boto3_session=session
        )
    except Exception as err:
        print(err)

In [29]:
wr.catalog.databases(boto3_session=session)


,Database,Description
0,crypto_data,OHLC data + Tweets
1,default,Default Hive database
2,master_project,this is a test database


In [33]:
wr.catalog.tables(database="crypto_data",boto3_session=session)

,Database,Table,Description,TableType,Columns,Partitions
0,crypto_data,ohlc_data,test_table,EXTERNAL_TABLE,"time, high, low, open, volumefrom, volumeto, c...",partition_column


In [31]:
# columns_types_dict = {
#     'time' : 'bigint',
#     'high' : 'double',
#     'low'  : 'double',
#     'open' : 'double',
#     'volumefrom'  : 'double',
#     'volumeto' : 'double',
#     'close'  : 'double',
#     'conversiontype' : 'string',
#     'conversionsymbol' : 'string',
#     'coin_currency'  : 'string',
# }

# wr.catalog.create_parquet_table(
#     database='crypto_data',
#     table='ohlc_data',
#     path=f's3://{bucket}/data/crypto_data/ohlc_data',
#     columns_types=columns_types_dict,
#     partitions_types={'partition_column' : 'string'},
#     compression='snappy',
#     description='test_table',
#     boto3_session=session
# )

In [32]:
%time
write_to_glue(df_test, path_table=f"s3://{bucket}/data/crypto_data/ohlc_data",
            database_name="crypto_data", table_name='ohlc_data',
            col_partition=['partition_column'])

CPU times: total: 0 ns
Wall time: 0 ns


In [34]:
params_tweets = {
        'Bucket' : bucket,
        'Prefix' : "data/daily_tweets/"
        }

In [35]:
def get_latest_tweets(operation_parameters, s3=None):
    """
    List files in specific S3 URL
    """
    if not s3:
        s3 = boto3.client('s3')
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for content in page.get('Contents'):
            if content.get('LastModified') > utc.localize(datetime.now() - timedelta(days=4)):
                yield content.get('Key')

In [36]:
tweets = []
for key in get_latest_tweets(params_tweets, s3_client):
    tmp = get_json_s3(bucket, key, s3_client)
    tweets.extend(tmp)


In [37]:
df_tweets = pd.DataFrame(tweets)

In [38]:
df_tweets['partition_column'] = pd.to_datetime(df_tweets['created_at']).dt.strftime('%Y-%m-%d')
df_tweets

,id,created_at,ceil_datetime,month,weekday,day,hour,minute,QUERY,ticker,text,favorite_count,result_type,user_name,followers_count,retweet_count,partition_column
0,1553003298109755392,2022-07-29 13:03:34,2022-07-29 13:15:00,7,5,29,13,15,bitcoin,BTC,Looking forward to giving one of my followers ...,2413,popular,MartiniGuyYT,390472,2052,2022-07-29
1,1552744653455335425,2022-07-28 19:55:48,2022-07-28 20:00:00,7,4,28,20,0,bitcoin,BTC,"JUST IN: ""I buy #Bitcoin for the benefit of my...",3206,popular,BitcoinMagazine,2584271,634,2022-07-28
2,1553005848062038017,2022-07-29 13:13:42,2022-07-29 13:15:00,7,5,29,13,15,bitcoin,BTC,JUST IN: German retailer giant REWE installs #...,2877,popular,BitcoinMagazine,2584271,601,2022-07-29
3,1553168472267378688,2022-07-29 23:59:54,2022-07-30 00:00:00,7,6,30,0,0,bitcoin,BTC,$BTC Shorted(Sell) [23:59:54 UTC]\n23767.60 ...,0,recent,xCryptoAlert,28,0,2022-07-29
4,1553168467721097216,2022-07-29 23:59:53,2022-07-30 00:00:00,7,6,30,0,0,bitcoin,BTC,RT @AurelienOhayon: #BITCOIN IS READY FOR A MA...,0,recent,cbin10etin1,51,33,2022-07-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37646,1553848148619141120,2022-07-31 21:00:42,2022-07-31 21:15:00,7,0,31,21,15,Litecoin,LTC,RT @PhibbiX: 250$ Litecoin Giveaway \n2 Winner...,0,recent,adhyrhiry,9,330,2022-07-31
37647,1553848127492427777,2022-07-31 21:00:37,2022-07-31 21:15:00,7,0,31,21,15,Litecoin,LTC,The current LTCBTC ratio is 0.00260400\n\nLite...,0,recent,masterltcbtcbot,661,0,2022-07-31
37648,1553848099793158144,2022-07-31 21:00:30,2022-07-31 21:15:00,7,0,31,21,15,Litecoin,LTC,Bitcoin - $23821.0 (23324.0)\nEthereum - ...,0,recent,CryptoBotPrice,188,0,2022-07-31
37649,1553848092063055878,2022-07-31 21:00:28,2022-07-31 21:15:00,7,0,31,21,15,Litecoin,LTC,@mirandalambert @MuttNation @thespringpoint #v...,2,recent,Litecoin_______,692,0,2022-07-31


In [40]:
df_tweets.dtypes

id                   int64
created_at          object
ceil_datetime       object
month                int64
weekday              int64
day                  int64
hour                 int64
minute               int64
QUERY               object
ticker              object
text                object
favorite_count       int64
result_type         object
user_name           object
followers_count      int64
retweet_count        int64
partition_column    object
dtype: object

In [41]:
columns_types_dict = {
    'id' : 'bigint',
    'created_at': 'date',
    'ceil_datetime': 'date',
    'month': 'int',
    'weekday': 'int',
    'day': 'int',
    'hour': 'int',
    'minute': 'int',
    'QUERY': 'string',
    'ticker': 'string',
    'text': 'string',
    'favorite_count': 'int',
    'result_type': 'string',
    'user_name': 'string',
    'followers_count': 'int',
    'retweet_count': 'int',
}

wr.catalog.create_parquet_table(
    database='crypto_data',
    table='tweets_data',
    path=f's3://{bucket}/data/crypto_data/tweets_data',
    columns_types=columns_types_dict,
    partitions_types={'partition_column' : 'string'},
    compression='snappy',
    description='tweet_test_table',
    boto3_session=session
)

In [42]:
%time
write_to_glue(df_tweets, path_table=f"s3://{bucket}/data/crypto_data/tweets_data",
            database_name="crypto_data", table_name='tweets_data',
            col_partition=['partition_column'])

CPU times: total: 0 ns
Wall time: 0 ns
